In [29]:
import os
import io
import gc
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Load the input data

In [12]:
dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32',
}

In [13]:
cols = ['ip','app','device','os', 'channel', 'click_time', 'is_attributed']

In [23]:
frm=0
to = 10000
print('loading train data...')
train_df = pd.read_csv("data/partial_data.csv", parse_dates=['click_time'], skiprows=range(1,frm), nrows=to-frm, dtype=dtypes, 
                       usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])
train_df.head(3)

loading train data...


,ip,app,device,os,channel,click_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,0
1,17357,3,1,19,379,2017-11-06 14:33:34,0
2,35810,3,1,13,379,2017-11-06 14:34:12,0


In [24]:
print('Extracting new features...')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['month'] = pd.to_datetime(df.click_time).dt.month.astype('uint8')
train_df['year'] = pd.to_datetime(df.click_time).dt.year.astype('uint8')
gc.collect()
train_df.head(3)

Extracting new features...


,ip,app,device,os,channel,click_time,is_attributed,hour,day,month,year
0,83230,3,1,13,379,2017-11-06 14:32:21,0,14,6,11,225
1,17357,3,1,19,379,2017-11-06 14:33:34,0,14,6,11,225
2,35810,3,1,13,379,2017-11-06 14:34:12,0,14,6,11,225


In [25]:
naddfeat=9
for i in range(0,naddfeat):
    if i==0: selcols=['ip', 'channel']; QQ=4;
    if i==1: selcols=['ip', 'device', 'os', 'app']; QQ=5;
    if i==2: selcols=['ip', 'day', 'hour']; QQ=4;
    if i==3: selcols=['ip', 'app']; QQ=4;
    if i==4: selcols=['ip', 'app', 'os']; QQ=4;
    if i==5: selcols=['ip', 'device']; QQ=4;
    if i==6: selcols=['app', 'channel']; QQ=4;
    if i==7: selcols=['ip', 'os']; QQ=5;
    if i==8: selcols=['ip', 'device', 'os', 'app']; QQ=4;
    print('selcols',selcols,'QQ',QQ)

    filename='X%d_%d_%d.csv'%(i,frm,to)

    if os.path.exists(filename):
        if QQ==5: 
            gp=pd.read_csv(filename,header=None)
            train_df['X'+str(i)]=gp
        else: 
            gp=pd.read_csv(filename)
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
    else:
        if QQ==0:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].count().reset_index().\
                rename(index=str, columns={selcols[len(selcols)-1]: 'X'+str(i)})
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
        if QQ==1:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].mean().reset_index().\
                rename(index=str, columns={selcols[len(selcols)-1]: 'X'+str(i)})
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
        if QQ==2:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].var().reset_index().\
                rename(index=str, columns={selcols[len(selcols)-1]: 'X'+str(i)})
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
        if QQ==3:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].skew().reset_index().\
                rename(index=str, columns={selcols[len(selcols)-1]: 'X'+str(i)})
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
        if QQ==4:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].nunique().reset_index().\
                rename(index=str, columns={selcols[len(selcols)-1]: 'X'+str(i)})
            train_df = train_df.merge(gp, on=selcols[0:len(selcols)-1], how='left')
        if QQ==5:
            gp = train_df[selcols].groupby(by=selcols[0:len(selcols)-1])[selcols[len(selcols)-1]].cumcount()
            train_df['X'+str(i)]=gp.values
        gp.to_csv(filename,index=False)
    del gp
    gc.collect()
train_df.head(3)

('selcols', ['ip', 'channel'], 'QQ', 4)
('selcols', ['ip', 'device', 'os', 'app'], 'QQ', 5)
('selcols', ['ip', 'day', 'hour'], 'QQ', 4)
('selcols', ['ip', 'app'], 'QQ', 4)
('selcols', ['ip', 'app', 'os'], 'QQ', 4)
('selcols', ['ip', 'device'], 'QQ', 4)
('selcols', ['app', 'channel'], 'QQ', 4)
('selcols', ['ip', 'os'], 'QQ', 5)
('selcols', ['ip', 'device', 'os', 'app'], 'QQ', 4)


,ip,app,device,os,channel,click_time,is_attributed,hour,day,month,year,X0,X1,X2,X3,X4,X5,X6,X7,X8
0,83230,3,1,13,379,2017-11-06 14:32:21,0,14,6,11,225,1,0,1,1,1,1,23,0,1
1,17357,3,1,19,379,2017-11-06 14:33:34,0,14,6,11,225,1,0,2,1,2,1,23,0,1
2,35810,3,1,13,379,2017-11-06 14:34:12,0,14,6,11,225,1,0,1,1,1,1,23,0,1


In [26]:
for i in range(naddfeat):
    print i, set(train_df['X'+str(i)])

0 set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 21, 22, 30, 31, 32, 39])
1 set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
2 set([1, 2, 3])
3 set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18])
4 set([1, 2, 3, 4, 5, 6, 7, 8])
5 set([1, 2, 3, 4])
6 set([1, 2, 3, 4, 5, 7, 9, 12, 15, 17, 18, 19, 23])
7 set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64])
8 set([1, 2, 3, 4, 5, 6, 7, 8, 9, 11])


In [28]:
print('doing nextClick')
predictors=[]

new_feature = 'nextClick'
filename='nextClick_%d_%d.csv'%(frm,to)

if os.path.exists(filename):
    print('loading from save file')
    QQ=pd.read_csv(filename).values
else:
    D=2**26
    train_df['category'] = (train_df['ip'].astype(str) + "_" + train_df['app'].astype(str) + "_" + train_df['device'].astype(str) \
        + "_" + train_df['os'].astype(str)).apply(hash) % D
    click_buffer= np.full(D, 3000000000, dtype=np.uint32)

    train_df['epochtime']= train_df['click_time'].astype(np.int64) // 10 ** 9
    next_clicks= []
    for category, t in zip(reversed(train_df['category'].values), reversed(train_df['epochtime'].values)):
        next_clicks.append(click_buffer[category]-t)
        click_buffer[category]= t
    del(click_buffer)
    QQ= list(reversed(next_clicks))

    print('saving')
    pd.DataFrame(QQ).to_csv(filename,index=False)

train_df[new_feature] = QQ
predictors.append(new_feature)

train_df[new_feature+'_shift'] = pd.DataFrame(QQ).shift(+1).values
predictors.append(new_feature+'_shift')

del QQ
gc.collect()

print('grouping by ip-day-hour combination...')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_tcount'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
del gp
gc.collect()

print('grouping by ip-app combination...')
gp = train_df[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp
gc.collect()

print('grouping by ip-app-os combination...')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

# Adding features with var and mean hour (inspired from nuhsikander's script)
print('grouping by : ip_day_chl_var_hour')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_tchan_count'})
train_df = train_df.merge(gp, on=['ip','day','channel'], how='left')
del gp
gc.collect()

print('grouping by : ip_app_os_var_hour')
gp = train_df[['ip','app', 'os', 'hour']].groupby(by=['ip', 'app', 'os'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_os_var'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

print('grouping by : ip_app_channel_var_day')
gp = train_df[['ip','app', 'channel', 'day']].groupby(by=['ip', 'app', 'channel'])[['day']].var().reset_index().rename(index=str, columns={'day': 'ip_app_channel_var_day'})
train_df = train_df.merge(gp, on=['ip','app', 'channel'], how='left')
del gp
gc.collect()

print('grouping by : ip_app_chl_mean_hour')
gp = train_df[['ip','app', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
print("merging...")
train_df = train_df.merge(gp, on=['ip','app', 'channel'], how='left')
del gp
gc.collect()

print("vars and data type: ")
train_df.info()
train_df['ip_tcount'] = train_df['ip_tcount'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

gc.collect()
train_df.head(3)

doing nextClick
saving
grouping by ip-day-hour combination...
grouping by ip-app combination...
grouping by ip-app-os combination...
grouping by : ip_day_chl_var_hour
grouping by : ip_app_os_var_hour
grouping by : ip_app_channel_var_day
grouping by : ip_app_chl_mean_hour
merging...
vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 31 columns):
ip                          10000 non-null uint32
app                         10000 non-null uint16
device                      10000 non-null uint16
os                          10000 non-null uint16
channel                     10000 non-null uint16
click_time                  10000 non-null datetime64[ns]
is_attributed               10000 non-null uint8
hour                        10000 non-null uint8
day                         10000 non-null uint8
month                       10000 non-null uint8
year                        10000 non-null uint8
X0                          10000 n

,ip,app,device,os,channel,click_time,is_attributed,hour,day,month,...,epochtime,nextClick,nextClick_shift,ip_tcount,ip_app_count,ip_app_os_count,ip_tchan_count,ip_app_os_var,ip_app_channel_var_day,ip_app_channel_mean_hour
0,83230,3,1,13,379,2017-11-06 14:32:21,0,14,6,11,...,1509978741,1490021259,NaN,1,1,1,NaN,NaN,NaN,14.0
1,17357,3,1,19,379,2017-11-06 14:33:34,0,14,6,11,...,1509978814,1490021186,1.490021e+09,1,2,1,0.5,NaN,0.0,14.5
2,35810,3,1,13,379,2017-11-06 14:34:12,0,14,6,11,...,1509978852,1490021148,1.490021e+09,1,1,1,NaN,NaN,NaN,14.0


In [30]:
train_df.head(3)

,ip,app,device,os,channel,click_time,is_attributed,hour,day,month,year,X0,X1,X2,X3,X4,X5,X6,X7,X8,category,epochtime,nextClick,nextClick_shift,ip_tcount,ip_app_count,ip_app_os_count,ip_tchan_count,ip_app_os_var,ip_app_channel_var_day,ip_app_channel_mean_hour
0,83230,3,1,13,379,2017-11-06 14:32:21,0,14,6,11,225,1,0,1,1,1,1,23,0,1,6219339,1509978741,1490021259,NaN,1,1,1,NaN,NaN,NaN,14.0
1,17357,3,1,19,379,2017-11-06 14:33:34,0,14,6,11,225,1,0,2,1,2,1,23,0,1,64430138,1509978814,1490021186,1.490021e+09,1,2,1,0.5,NaN,0.0,14.5
2,35810,3,1,13,379,2017-11-06 14:34:12,0,14,6,11,225,1,0,1,1,1,1,23,0,1,39185248,1509978852,1490021148,1.490021e+09,1,1,1,NaN,NaN,NaN,14.0


In [33]:
predictors.extend(['app','device','os', 'channel', 'hour', 'day', 'month', 'year'
                  'ip_tcount', 'ip_tchan_count', 'ip_app_count',
                  'ip_app_os_count', 'ip_app_os_var',
                  'ip_app_channel_var_day','ip_app_channel_mean_hour'])
categorical = ['app', 'device', 'os', 'channel', 'hour', 'day']
for i in range(0,naddfeat):
    predictors.append('X'+str(i))

print('predictors',predictors)

('predictors', ['nextClick', 'nextClick_shift', 'app', 'device', 'os', 'channel', 'hour', 'day', 'ip_tcount', 'ip_tchan_count', 'ip_app_count', 'ip_app_os_count', 'ip_app_os_var', 'ip_app_channel_var_day', 'ip_app_channel_mean_hour', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'app', 'device', 'os', 'channel', 'hour', 'day', 'month', 'yearip_tcount', 'ip_tchan_count', 'ip_app_count', 'ip_app_os_count', 'ip_app_os_var', 'ip_app_channel_var_day', 'ip_app_channel_mean_hour', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'])
